In [ ]:
# !pip install --user pymorphy2[fast]

In [ ]:
# !pip install --user annoy

In [ ]:
# !pip install --user transformers

In [1]:
import string
import nltk
import annoy
import numpy as np
import tensorflow as tf
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from tqdm import notebook
from transformers import TFAutoModel, AutoTokenizer

In [2]:
!head -5 ./data/prepared_answers.txt

"head" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [ ]:
# assert True

# #Small preprocess of the answers

# question = None
# written = False

# with open("./data/prepared_answers.txt", "w", encoding="utf-8") as fout:
#     with open("./data/Otvety.txt", "r", encoding="utf-8") as fin:
#         for line in notebook.tqdm(fin):
#             if line.startswith("---"):
#                 written = False
#                 continue
#             if not written and question is not None:
#                 fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
#                 written = True
#                 question = None
#                 continue
#             if not written:
#                 question = line.strip()
#                 continue

In [3]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return ' '.join(spls)

In [4]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru") + nltk.corpus.stopwords.words('russian'))
exclude = set(string.punctuation)

In [5]:
bert = TFAutoModel.from_pretrained("setu4993/LaBSE")
tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at setu4993/LaBSE.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [3]:
q_len = []
a_len = []
with open("./data/prepared_answers.txt", "r", encoding="utf-8") as f:
    counter = 0
    for line in notebook.tqdm(f):
        spls = line.split("\t")
        q_len.append(len(spls[0].split()))
        a_len.append(len(spls[1].split()))

In [4]:
np.median(q_len), np.mean(q_len), np.quantile(q_len, 0.75), np.quantile(q_len, 0.85), np.quantile(q_len, 0.95), np.max(q_len)

(12.0, 20.454707214215595, 21.0, 31.0, 63.0, 862)

In [5]:
import json

In [6]:
bert_index = annoy.AnnoyIndex(768 , 'angular')

index_map = {}
counter = 0

In [22]:
# from threading import Thread

In [23]:
def tokenize(line, counter):
    spls = line.split("\t")
    if len(spls[0].split()) < 2 or len(spls[1].split()) < 3 or len(spls[0].split()) > 15:
        return counter

    index_map[counter] = spls[1]
    question = preprocess_txt(spls[0])

    tok = tokenizer(question, return_token_type_ids=False, return_tensors='tf')
    out_state = bert(**tok)[1].numpy()[0]
    bert_index.add_item(counter, out_state)
    counter += 1
    return counter

In [30]:
%%time

with open("./data/prepared_answers.txt", "r", encoding="utf-8") as f:
    for line in notebook.tqdm(f):
#         thr = Thread(target=tokenize, args=(line, counter))
#         thr.start()
        spls = line.split("\t")
        if len(spls[0].split()) < 2 or len(spls[1].split()) < 3 or len(spls[0].split()) > 15:
            continue
        
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        tok = tokenizer(question, return_token_type_ids=False, return_tensors='tf')
        out_state = bert(**tok)[1].numpy()[0]
        bert_index.add_item(counter, out_state)
        counter += 1
        

bert_index.build(10)
bert_index.save('./data/bert_index')

with open('./data/index_map.json', 'w') as f:
    json.dump(index_map, f, indent=4, ensure_ascii=False)

UnicodeEncodeError: 'charmap' codec can't encode character '\u2666' in position 1: character maps to <undefined>

In [8]:
import pickle

In [39]:
with open('./data/index_map.pkl', 'wb') as f:
    pickle.dump(index_map, f)

In [7]:
bert_index.load('./data/bert_index')

True

In [9]:
with open('./data/index_map.pkl', 'rb') as f:
    index_map = pickle.load(f)

In [85]:
def get_response(question, index, index_map):
    question = preprocess_txt(question)
    tok = tokenizer(question, return_token_type_ids=False, return_tensors='tf')
    vector = bert(**tok)[1].numpy()[0]
    answers, distances = index.get_nns_by_vector(vector, 10, include_distances=True)
#     answer_idx = np.argmax(distances)
#     return distances
#     return [(dist, index_map[i]) for i, dist in zip(answers, distances)]
#     return index_map[answers[answer_idx]]
    return index_map[answers[distances.index(np.random.choice(distances[-3:]))]]

In [86]:
get_response('куда сходить вечером?', bert_index, index_map)

'Загорать и купаться конечно же!!Что водоёмов нет никаких поблизости?. \n'

In [82]:
a.index(np.random.choice(a[-3:]))

7